In [1]:
import os
import torch
import pandas as pd
import oracledb
import re
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline, BitsAndBytesConfig
from huggingface_hub import login
from dotenv import load_dotenv

load_dotenv()

device = "cuda" if torch.cuda.is_available() else "cpu"

# Ensure less fragmentation
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "expandable_segments:True"

login(token=os.getenv("HF_TOKEN"))
print(device)

/home/ritik26/Desktop/Internship_2/virtual_environment/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Note: Environment variable`HF_TOKEN` is set and is the current active token independently from the token you've just configured.


cuda


In [3]:
model_id = "RedHatAI/Mistral-7B-Instruct-v0.3-quantized.w4a16"

# Load tokenizer safely
tokenizer = AutoTokenizer.from_pretrained(
    model_id,
    trust_remote_code=True,
    use_fast=False  # Prevent tokenizer enum issues
)

# Load model WITHOUT BitsAndBytesConfig since it's already quantized
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    device_map="auto",              # Automatically use GPU if available
    trust_remote_code=True,
    torch_dtype=torch.float16       # Match quantized dtype
)

# Load text-generation pipeline
pipe = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    device_map="auto",
    torch_dtype=torch.float16
)

`loss_type=None` was set in the config but it is unrecognised.Using the default loss: `ForCausalLMLoss`.
Device set to use cuda:0


In [4]:
# Database configuration
DB_CONFIG = {
    "host": os.getenv("DB_HOST"),
    "port": os.getenv("DB_PORT"),
    "service_name": os.getenv("DB_SERVICE"),
    "user": os.getenv("DB_USER"),
    "password": os.getenv("DB_PASSWORD")
}

# Build the DSN
dsn = f"{DB_CONFIG['host']}:{DB_CONFIG['port']}/{DB_CONFIG['service_name']}"

dsn = f"{DB_CONFIG['host']}:{DB_CONFIG['port']}/{DB_CONFIG['service_name']}"
connection = oracledb.connect(user=DB_CONFIG['user'], password=DB_CONFIG['password'], dsn=dsn)
cursor = connection.cursor()

In [5]:
def run_sql(query: str):
    try:
        cursor.execute(query)
        columns = [col[0] for col in cursor.description]
        data = cursor.fetchall()
        df = pd.DataFrame(data, columns=columns)
        return df.head(5).to_markdown(index=False)
    except Exception as e:
        return f"[SQL Error] {str(e)}"

In [6]:
def get_schema_overview():
    query = "SELECT table_name FROM all_tables WHERE owner = 'PROP'"
    cursor.execute(query)
    tables = [row[0] for row in cursor.fetchall()]

    metadata = ""
    for table in tables:
        try:
            cursor.execute(f"SELECT column_name, data_type FROM all_tab_columns WHERE table_name = '{table}'")
            cols = cursor.fetchall()
            col_str = ", ".join(f"{col[0]} ({col[1]})" for col in cols)
            metadata += f"Table: {table}\nColumns: {col_str}\n\n"
        except:
            continue
    return metadata.strip()

In [7]:
# Generate exploratory prompt
schema_summary = get_schema_overview()
initial_prompt = f"""
You are a data analyst agent connected to an Oracle SQL database.
Below is the schema overview of the database:

{schema_summary}

Your task is to:
You are not allowed to manipulate the database directly. Never use INSERT, UPDATE, DELETE, or DROP statements.
1. Detect duplicate rows or dummy data in each table.
2. Identify relationships between tables (primary/foreign keys).
3. Describe the distribution and structure of data in each table.
4. Explain what each table most likely stores.
5. Suggest any insights or anomalies.
6. Provide SQL queries to extract relevant data.
7. Provide explanation after analyzing each table and its columns.
8. If you encounter any errors, provide the SQL error message.
9. Also include the database name before the table name in the SQL queries. (eg: PROP.table_name)
10. Run with join queries too, to get more insights on the relationships between tables.

You may request SQL queries. Begin with the first one.
Only return the SQL query to be executed in a single code block.
"""

## Schema Summary:

In [9]:
with open("initial_prompt.txt", "w") as f:
    f.write(initial_prompt)

In [10]:
executed_queries = set()
for _ in range(2):
    response = pipe(
        initial_prompt,
        max_new_tokens=512,
        do_sample=True,
        temperature=0.7,
        pad_token_id=tokenizer.eos_token_id
    )[0]["generated_text"]

    print("LLM Output:\n", response)

    match = re.search(r"```sql\s+(.*?)```", response, re.DOTALL | re.IGNORECASE)
    if not match:
        match = re.search(r"(SELECT .*?;)", response, re.IGNORECASE | re.DOTALL)
    if not match:
        print("[No SQL found]")
        break

    query = match.group(1).strip().rstrip(";")
    if query in executed_queries:
        print("[Repeated query detected, exiting]")
        break
    executed_queries.add(query)

    output = run_sql(query)
    print("Query Result Preview:\n", output)

    initial_prompt += f"\n\nQuery: {query}\nResult Preview:\n{output}\n\nNext?"

OutOfMemoryError: CUDA out of memory. Tried to allocate 2.93 GiB. GPU 0 has a total capacity of 7.62 GiB of which 1.65 GiB is free. Including non-PyTorch memory, this process has 5.78 GiB memory in use. Of the allocated memory 5.62 GiB is allocated by PyTorch, and 44.13 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)